In [1]:
import tensorflow as tf
s = tf.InteractiveSession()

# 1st assignment: MSE

To implement mean squared error in tensorflow.

In [2]:
with tf.name_scope("MSE"):
    y_true = tf.placeholder("float32", shape=(None,), name="y_true")
    y_predicted = tf.placeholder("float32", shape=(None,), name="y_predicted")
    mse = tf.reduce_mean((y_true - y_predicted)**2)
def compute_mse(vector1, vector2):
    return mse.eval({y_true: vector1, y_predicted: vector2})

In [3]:
# dummy = np.arange(5).astype('float32')
vector1 = [1,1,3]
vector2 = [1,1,1]
print(vector1, vector2)
compute_mse(vector1,vector2)

[1, 1, 3] [1, 1, 1]


1.3333334

# 2nd assignment: Logistic regression
To implement the logistic regression

Plan:
* Use a shared variable for weights
* Use a matrix placeholder for `X`
 
We shall train on a two-class MNIST dataset
* please note that target `y` are `{0,1}` and not `{-1,1}` as in some formulae

In [4]:
from sklearn.datasets import load_digits
mnist = load_digits(2)

X, y = mnist.data, mnist.target

print("y [shape - %s]:" % (str(y.shape)), y[:10])
print("X [shape - %s]:" % (str(X.shape)))
print('X:\n',X[:3,:10])
print('y:\n',y[:10])

y [shape - (360,)]: [0 1 0 1 0 1 0 0 1 1]
X [shape - (360, 64)]:
X:
 [[  0.   0.   5.  13.   9.   1.   0.   0.   0.   0.]
 [  0.   0.   0.  12.  13.   5.   0.   0.   0.   0.]
 [  0.   0.   1.   9.  15.  11.   0.   0.   0.   0.]]
y:
 [0 1 0 1 0 1 0 0 1 1]


It's your turn now!
Just a small reminder of the relevant math:

$$
P(y=1|X) = \sigma(X \cdot W + b)
$$
$$
\text{loss} = -\log\left(P\left(y_\text{predicted} = 1\right)\right)\cdot y_\text{true} - \log\left(1 - P\left(y_\text{predicted} = 1\right)\right)\cdot\left(1 - y_\text{true}\right)
$$

$\sigma(x)$ is available via `tf.nn.sigmoid` and matrix multiplication via `tf.matmul`

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)
X_train.shape, y_train.shape, X_test.shape

((270, 64), (270,), (90, 64))

In [6]:
# Model parameters - weights and bias
weights = tf.Variable(tf.random_normal([X.shape[1],1], stddev=0.35),
                      name="weights") 
b = tf.Variable(0, dtype='float32', name="biases")

In [7]:
weights, b

(<tf.Variable 'weights:0' shape=(64, 1) dtype=float32_ref>,
 <tf.Variable 'biases:0' shape=() dtype=float32_ref>)

In [8]:
# Placeholders for the input data
input_X = tf.placeholder('float32', shape=(None,None))
input_y = tf.placeholder('float32', shape=(None, ))
input_X, input_y

(<tf.Tensor 'Placeholder:0' shape=(?, ?) dtype=float32>,
 <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=float32>)

In [9]:
# The model code

# Compute a vector of predictions, resulting shape should be [input_X.shape[0],]
# This is 1D, if you have extra dimensions, you can  get rid of them with tf.squeeze .
# Don't forget the sigmoid.
a = tf.matmul(input_X, weights)
predicted_y =  tf.nn.sigmoid(tf.squeeze(a)+b)

# Loss. Should be a scalar number - average loss over all the objects
loss = tf.reduce_mean(-tf.log(predicted_y+1e-07)*input_y-tf.log(1-predicted_y+1e-07)*(1 - input_y))

# See above for an example. tf.train.*Optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(
    loss, var_list=(weights,b))

In [10]:
predicted_y, loss, optimizer, a

(<tf.Tensor 'Sigmoid:0' shape=<unknown> dtype=float32>,
 <tf.Tensor 'Mean:0' shape=<unknown> dtype=float32>,
 <tf.Operation 'GradientDescent' type=NoOp>,
 <tf.Tensor 'MatMul:0' shape=(?, 1) dtype=float32>)

A test to help with the debugging

In [11]:
import numpy as np

my_scalar = tf.placeholder('float32')
my_vector = tf.placeholder('float32', [None])

# Warning! Trying to understand the meaning of that function may result in permanent brain damage
weird_psychotic_function = tf.reduce_mean(
    (my_vector+my_scalar)**(1+tf.nn.moments(my_vector,[0])[1]) + 
    1./ tf.atan(my_scalar))/(my_scalar**2 + 1) + 0.01*tf.sin(
    2*my_scalar**1.5)*(tf.reduce_sum(my_vector)* my_scalar**2
                      )*tf.exp((my_scalar-4)**2)/(
    1+tf.exp((my_scalar-4)**2))*(1.-(tf.exp(-(my_scalar-4)**2)
                                    )/(1+tf.exp(-(my_scalar-4)**2)))**2

validation_weights = 1e-3 * np.fromiter(map(lambda x:
        s.run(weird_psychotic_function, {my_scalar:x, my_vector:[1, 0.1, 2]}),
                                   0.15 * np.arange(1, X.shape[1] + 1)),
                                   count=X.shape[1], dtype=np.float32)[:, np.newaxis]

# Compute predictions for given weights and bias
prediction_validation = s.run(
    predicted_y, {
    input_X: X,
    weights: validation_weights,
    b: 1e-1})


In [12]:
# Load the reference values for the predictions
validation_true_values = np.loadtxt("D:\\Downloads\\validation_predictons.txt"),
prediction_validation.shape == (X.shape[0],), \
       "Predictions must be a 1D array with length equal to the number " \
       "of examples in input_X"


(True,
 'Predictions must be a 1D array with length equal to the number of examples in input_X')

In [13]:
np.allclose(validation_true_values, prediction_validation)

True

In [14]:
loss_validation = s.run(
        loss, {
            input_X: X[:100],
            input_y: y[-100:],
            weights: validation_weights+1.21e-3,
            b: -1e-1})

In [15]:
np.allclose(loss_validation, 0.728689)

True

In [16]:
from sklearn.metrics import roc_auc_score
s.run(tf.global_variables_initializer())
for i in range(5):
    s.run(optimizer, {input_X: X_train, input_y: y_train})
    loss_i = s.run(loss, {input_X: X_train, input_y: y_train})
    print("loss at iter %i:%.4f" % (i, loss_i))
    print("train auc:", roc_auc_score(y_train, s.run(predicted_y, {input_X:X_train})))
    print("test auc:", roc_auc_score(y_test, s.run(predicted_y, {input_X:X_test})))

loss at iter 0:1.7502
train auc: 0.997200263505
test auc: 0.997529644269
loss at iter 1:3.2488
train auc: 0.92803030303
test auc: 0.945652173913
loss at iter 2:3.1453
train auc: 1.0
test auc: 1.0
loss at iter 3:0.0784
train auc: 1.0
test auc: 1.0
loss at iter 4:0.0349
train auc: 1.0
test auc: 1.0


In [17]:
test_weights = 1e-3 * np.fromiter(map(lambda x:
    s.run(weird_psychotic_function, {my_scalar:x, my_vector:[1, 2, 3]}),
                               0.1 * np.arange(1, X.shape[1] + 1)),
                               count=X.shape[1], dtype=np.float32)[:, np.newaxis]

First, test prediction and loss computation. This part doesn't require a fitted model.

In [18]:
prediction_test = s.run(
    predicted_y, {
    input_X: X,
    weights: test_weights,
    b: 1e-1})

In [19]:
prediction_test.shape == (X.shape[0],),\
       "Predictions must be a 1D array with length equal to the number " \
       "of examples in X_test"

(True,
 'Predictions must be a 1D array with length equal to the number of examples in X_test')

In [20]:
loss_test = s.run(
    loss, {
        input_X: X[:100],
        input_y: y[-100:],
        weights: test_weights+1.21e-3,
        b: -1e-1})
# Yes, the X/y indices mistmach is intentional

In [21]:
loss_test, roc_auc_score(y_test, s.run(predicted_y, {input_X:X_test}))

(0.77501142, 1.0)